# Examples

This notebook contains a few examples on how to use `pelage`. The idea is to illustrate
what the main features with an succession of checks / transformation. We use here a simple
example: [the MPG dataset](https://github.com/mwaskom/seaborn-data), loaded using the seaborn utility function.


## Imports

In [9]:
import polars as pl
import seaborn as sns

import pelage as plg

data = pl.DataFrame(sns.load_dataset("mpg"))
data.head()

mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
f64,i64,f64,f64,i64,f64,i64,str,str
18.0,8,307.0,130.0,3504,12.0,70,"""usa""","""chevrolet chev…"
15.0,8,350.0,165.0,3693,11.5,70,"""usa""","""buick skylark …"
18.0,8,318.0,150.0,3436,11.0,70,"""usa""","""plymouth satel…"
16.0,8,304.0,150.0,3433,12.0,70,"""usa""","""amc rebel sst"""
17.0,8,302.0,140.0,3449,10.5,70,"""usa""","""ford torino"""


## Basic data transformations

In the following example, we perform some basic checks followed by a simple data 
transformation and finally checking for the presence of outliers. 

In [10]:
average_mileage_per_zone = (
    data.pipe(plg.has_no_nulls, ["origin", "cylinders", "model_year"])
    .pipe(plg.accepted_range, {"cylinders": (3, 8)})
    .pipe(plg.accepted_values, {"origin": ["usa", "europe", "japan"]})
    .filter(pl.col("model_year") >= 80)
    .group_by("origin", "cylinders", "model_year")
    .agg(
        n_distinct_models=pl.col("name").n_unique(),
        avg_mpg=pl.col("mpg").mean(),
    )
    .filter(pl.col("n_distinct_models") >= 3)
    .pipe(plg.column_is_within_n_std, ("avg_mpg", 3))
)
average_mileage_per_zone

origin,cylinders,model_year,n_distinct_models,avg_mpg
str,i64,i64,u32,f64
"""usa""",6,81,4,20.925
"""usa""",4,80,6,27.05
"""usa""",4,82,17,29.647059
"""japan""",4,81,10,34.59
"""japan""",4,80,11,36.709091
"""europe""",4,81,3,31.866667
"""usa""",6,82,3,28.333333
"""usa""",4,81,7,30.95
"""europe""",4,80,8,37.4


# Focus on the errors

## Error message

When the check fails, a PolarsAssertError is raise. The error message tends to provide 
a summarized view of the problem that occurred during the check. 

In [13]:
(
    data.pipe(
        plg.accepted_range,
        {"displacement": (50, 300), "horsepower": (50, 200)},
    )
)
# Generate a PolarsAssertError

PolarsAssertError: Details
shape: (104, 9)
┌──────┬───────────┬─────────────┬────────────┬───┬─────────────┬────────────┬────────┬────────────┐
│ mpg  ┆ cylinders ┆ displacemen ┆ horsepower ┆ … ┆ acceleratio ┆ model_year ┆ origin ┆ name       │
│ ---  ┆ ---       ┆ t           ┆ ---        ┆   ┆ n           ┆ ---        ┆ ---    ┆ ---        │
│ f64  ┆ i64       ┆ ---         ┆ f64        ┆   ┆ ---         ┆ i64        ┆ str    ┆ str        │
│      ┆           ┆ f64         ┆            ┆   ┆ f64         ┆            ┆        ┆            │
╞══════╪═══════════╪═════════════╪════════════╪═══╪═════════════╪════════════╪════════╪════════════╡
│ 18.0 ┆ 8         ┆ 307.0       ┆ 130.0      ┆ … ┆ 12.0        ┆ 70         ┆ usa    ┆ chevrolet  │
│      ┆           ┆             ┆            ┆   ┆             ┆            ┆        ┆ chevelle   │
│      ┆           ┆             ┆            ┆   ┆             ┆            ┆        ┆ malibu     │
│ 15.0 ┆ 8         ┆ 350.0       ┆ 165.0      ┆ 

## Investigating the cause of the failure

In addition to help theuser better understand the root cause of the check failure,
the error object also possesses as `df` attribute that can contains the identified 
values causing the check to
fail. 

Here is how to simply retrieve it without adding a `try/except` block. This allows us to print the error in a string format. 

In [ ]:
import sys

error = sys.last_value

print(error)

You can then manipulate a subset dataframe containing the elements that triggered the exception. Here we do a few manipulations to determine what are the values that are outside the specified boundaries as well as their relative importance within the dataset. 

In [15]:
(
    pl.DataFrame(error.df)  # This is only here to obtain syntax highlighting
    .select(pl.col("displacement", "horsepower"))
    .describe()
)

statistic,displacement,horsepower
str,f64,f64
"""count""",104.0,104.0
"""null_count""",0.0,0.0
"""mean""",334.221154,154.278846
"""std""",74.472899,37.102968
"""min""",68.0,46.0
"""25%""",305.0,140.0
"""50%""",350.0,150.0
"""75%""",360.0,175.0
"""max""",455.0,230.0
